In [107]:
! pip install pyodbc

In [108]:
! pip install unidecode

In [136]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/1push/Downloads/archive/urban_pluvial_flood_risk_dataset.csv")
df

segment_id             city_name    admin_ward   latitude   longitude  \
0     SEG-00001    Colombo, Sri Lanka  Borough East   6.920633   79.912600   
1     SEG-00002        Chennai, India        Ward D  13.076487   80.281774   
2     SEG-00003      Ahmedabad, India     Sector 12  23.019473   72.638578   
3     SEG-00004      Hong Kong, China     Sector 14  22.302602  114.078673   
4     SEG-00005  Durban, South Africa      Sector 5 -29.887602   30.911008   
...         ...                   ...           ...        ...         ...   
2958  SEG-02959         Paris, France        Ward B  48.872870    2.246250   
2959  SEG-02960       Shanghai, China     Sector 17  31.195529  121.435540   
2960  SEG-02961     Vancouver, Canada     Sector 12  49.162783 -123.037084   
2961  SEG-02962        Lagos, Nigeria        Zone V   6.504570    3.388571   
2962  SEG-02963          Osaka, Japan       Zone IV  34.740562  135.469263   

     catchment_id  elevation_m               dem_source       land_use  \
0         CAT-136          NaN     Copernicus_EEA-10_v5  Institutional   
1         CAT-049        -2.19     Copernicus_EEA-10_v5    Residential   
2         CAT-023        30.88                SRTM_3arc     Industrial   
3         CAT-168        24.28                SRTM_3arc    Residential   
4         CAT-171        35.70                SRTM_3arc     Industrial   
...           ...          ...                      ...            ...   
2958      CAT-036        30.46  Copernicus_GLO-30_v2023    Residential   
2959      CAT-050        -3.00                SRTM_3arc     Industrial   
2960      CAT-052        14.16  Copernicus_GLO-90_v2022          Green   
2961      CAT-092         7.00  Copernicus_GLO-30_v2023     Industrial   
2962      CAT-169        56.00                SRTM_3arc     Commercial   

     soil_group  drainage_density_km_per_km2  storm_drain_proximity_m  \
0           NaN                         4.27                    160.5   
1             D                         7.54                      NaN   
2             B                        11.00                    152.5   
3             B                         7.32                     37.0   
4             C                         4.50                    292.4   
...         ...                          ...                      ...   
2958          C                          NaN                     78.8   
2959          C                         5.54                      4.9   
2960          B                          NaN                    236.1   
2961          B                         8.74                    294.8   
2962          C                         4.44                     80.8   

     storm_drain_type rainfall_source  historical_rainfall_intensity_mm_hr  \
0           CurbInlet            ERA5                                 39.4   
1         OpenChannel            ERA5                                 56.8   
2         OpenChannel             IMD                                 16.3   
3             Manhole            ERA5                                 77.0   
4         OpenChannel            ERA5                                 20.8   
...               ...             ...                                  ...   
2958      GratedInlet             NaN                                 46.5   
2959      OpenChannel      LocalGauge                                 70.6   
2960      GratedInlet             NaN                                 27.7   
2961      OpenChannel            ERA5                                131.6   
2962      OpenChannel            ERA5                                 43.1   

      return_period_years                                 risk_labels  
0                      50                                     monitor  
1                      25  ponding_hotspot|low_lying|event_2025-05-02  
2                       5                                     monitor  
3                      10                                     monitor  
4    

In [139]:
#Extract city_name into city and country name
df[['city_name','country']] = df['city_name'].str.split(",",n = 1,expand = True)
df['city_name'] = df['city_name'].str.strip()
df['country'] = df['country'].str.strip()

In [141]:
df.isnull().sum()

segment_id                               0
city_name                                0
admin_ward                               0
latitude                                 0
longitude                                0
catchment_id                             0
elevation_m                            161
dem_source                               0
land_use                                 0
soil_group                             362
drainage_density_km_per_km2            284
storm_drain_proximity_m                239
storm_drain_type                       178
rainfall_source                        315
historical_rainfall_intensity_mm_hr      0
return_period_years                      0
risk_labels                              0
country                                  0
dtype: int64

In [215]:
#handling errors 

df['elevation_m'] = df['elevation_m'].replace(-3.0,np.nan)

df['soil_group'] = df['soil_group'].fillna('Unknown')

df['drainage_density_km_per_km2	']   = df['drainage_density_km_per_km2'].replace(0, np.nan)

df.storm_drain_proximity_m    = df.storm_drain_proximity_m.replace(0, np.nan)

df.rainfall_source = df.rainfall_source.fillna('Unknown')
df['storm_drain_type'] = df['storm_drain_type'].fillna('Unknown')


#handling Nan value

df['elevation_m'] = df.groupby('city_name')['elevation_m'].transform(lambda x: x.fillna(x.median()))

df['drainage_density_km_per_km2'] = df.groupby('city_name')['drainage_density_km_per_km2'].transform(lambda x: x.fillna(x.median()))


df['storm_drain_proximity_m'] = df.groupby('city_name')['storm_drain_proximity_m'].transform(lambda x : x.fillna(x.median()))
df.drop(columns = ['drainage_density_km_per_km2\t'],inplace = True)

df['risk_label1'] = df['risk_labels'].str.split('|',expand = True)[0]
df['risk_label1'].unique()

array(['monitor', 'ponding_hotspot', 'extreme_rain_history', 'low_lying',
       'sparse_drainage'], dtype=object)

In [217]:
# New classification Labels
def checker(value):
    if value == 'monitor':
        return 'low risk'
    elif value == 'ponding_hotspot':
        return 'high risk'
    elif value == 'extreme_rain_history':
        return 'high risk'
    elif value == 'low_lying':
        return 'low risk'
    else:
        return 'medium risk' 

In [219]:
df['risk_label1'] = df['risk_label1'].apply(checker)

In [221]:
df['risk_label1'].value_counts()

risk_label1
low risk       2443
high risk       395
medium risk     125
Name: count, dtype: int64